In [1]:
import json # will be needed for saving preprocessing details
import numpy as np # for data manipulation
import pandas as pd # for data manipulation
from sklearn.model_selection import train_test_split # will be used for data split
from sklearn.preprocessing import LabelEncoder # for preprocessing
from sklearn.ensemble import RandomForestClassifier # for training the algorithm
from sklearn.ensemble import ExtraTreesClassifier # for training the algorithm
import joblib # for saving algorithm and preprocessing objects
import sqlite3
from geopy.geocoders import Nominatim

In [2]:
conn = sqlite3.connect("../../db.sqlite3")
cur = conn.cursor()
cur.execute('''select * from sqlite_master''')
l = cur.fetchall()
#we will populate a list with the names of your tables
tbl_name_list = []
for sql_type, sql_name, tbl_name, rootpage, sql in l:
    if sql_type == 'table':
        tbl_name_list.append(sql_name)
df_account = pd.read_sql_query("SELECT * FROM account_account", conn).drop(
    ["password","is_admin","is_staff","is_superuser","is_active","email","username"], axis=1
)

df_account['location'] = df_account.streetNumber.map(str) + " , " + df_account.streetName + " , " + df_account.postcode + " , " + df_account.city + " , " + df_account.country
print(df_account['location'])
locator = Nominatim(user_agent="myGeocoder")
df_account['locatioan'] = locator.geocode(df_account['location'][0])
df_account['latitude'] = None
df_account['longitude'] = None
for i in range(len(df_account)):
    #print("{} , {}".format(i,df['location'][i]))

    if locator.geocode(df_account['location'][i]) == None:
        #print("{} , {}".format(i,df['location'][i].split(', ', 1)[1]))
        df_account['location'][i] = df_account['location'][i].split(', ', 1)[1]
        if locator.geocode(df_account['location'][i]) == None:
            df_account['location'][i] = df_account['location'][i].split(', ', 1)[1]
            if locator.geocode(df_account['location'][i]) == None:
                print()
            else:
                print("found 2 {}".format(i))
                location = locator.geocode(df_account['location'][i])
                print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))
                df_account['latitude'][i] = location.latitude
                df_account['longitude'][i] = location.longitude
        else:
            print("found 1 {}".format(i))
            location = locator.geocode(df_account['location'][i])
            print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))
            df_account['latitude'][i] = location.latitude
            df_account['longitude'][i] = location.longitude
    else:
        print("found 0 {}".format(i))
        location = locator.geocode(df_account['location'][i])
        print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))
        df_account['latitude'][i] = location.latitude
        df_account['longitude'][i] = location.longitude
df_account = df_account.drop(["city","country","streetName","streetNumber","location","locatioan"],axis=1)


df_click = pd.read_sql_query("SELECT * FROM company_click", conn)
df_company = pd.read_sql_query("SELECT * FROM company_company", conn).drop(["name"],axis=1)
df = df_company.merge(df_click,left_on="id",right_on="company_id")
df.dtypes
df = df.drop(['id_x','id_y'],axis=1).sort_values(by=['date_click'])
df = df.merge(df_account,left_on="user_id",right_on="id")
df_searched = pd.read_sql_query("SELECT * FROM personal_search_input", conn)
df_searched = df_searched.sort_values(by=['date_search'], ascending=False).drop_duplicates(subset=['user_id'])
df = df.merge(df_searched.drop(['id'],axis=1),left_on="user_id",right_on="user_id",how="left").sort_values(by=["date_click"])
df = df.drop(["user_id","ad_id","company_id","id"],axis=1)
df['date_click'] = pd.to_datetime(df['date_click'])
df['date_joined'] = pd.to_datetime(df['date_joined'])
df['last_login'] = pd.to_datetime(df['last_login'])
df['birthDate'] = pd.to_datetime(df['birthDate'])
df['date_search'] = pd.to_datetime(df['date_search'])
df_account

0             sNumber , sName , 54000 , Nancy , France
1             sNumber , sName , 54000 , Nancy , France
2             sNumber , sName , 54000 , Nancy , France
3             sNumber , sName , 54000 , Nancy , France
4             sNumber , sName , 54000 , Nancy , France
5    94 - 86 , Rue Sergent Blandan , 54000 , NANCY ...
Name: location, dtype: object


D:\program files\anaconda\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\program files\anaconda\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


found 2 0
Latitude = 48.69127075918049, Longitude = 6.181005123242174


D:\program files\anaconda\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\program files\anaconda\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


found 2 1
Latitude = 48.69127075918049, Longitude = 6.181005123242174
found 2 2
Latitude = 48.69127075918049, Longitude = 6.181005123242174
found 2 3
Latitude = 48.69127075918049, Longitude = 6.181005123242174
found 2 4
Latitude = 48.69127075918049, Longitude = 6.181005123242174
found 0 5
Latitude = 48.6722261, Longitude = 6.1723915


D:\program files\anaconda\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\program files\anaconda\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,date_joined,last_login,birthDate,postcode,latitude,longitude
0,1,2020-11-13 11:59:05.663783,2021-01-13 15:17:02.422077,2020-12-07 18:53:39.744654,54000,48.6913,6.18101
1,3,2020-11-14 14:27:55.360673,2021-01-13 15:16:47.734064,2020-12-07 18:53:39.744654,54000,48.6913,6.18101
2,4,2020-11-14 16:13:10.793579,2021-01-13 15:16:30.364840,2020-12-07 18:53:39.744654,54000,48.6913,6.18101
3,5,2020-11-14 16:42:23.882927,2021-01-20 00:12:38.036482,2020-12-07 18:53:39.744654,54000,48.6913,6.18101
4,6,2020-11-15 11:35:36.948280,2020-11-17 14:06:43.990779,2020-12-07 18:53:39.744654,54000,48.6913,6.18101
5,7,2020-12-07 18:54:40.475322,2021-01-22 16:42:11.306918,2020-12-02 00:00:00,54000,48.6722,6.17239


In [3]:
df["date_click_year"] = df["date_click"].dt.year
df["date_click_month"] = df["date_click"].dt.month
df["date_click_week"] = df["date_click"].dt.week
df["date_click_day"] = df["date_click"].dt.day
df["date_click_hour"] = df["date_click"].dt.hour
df["date_click_minute"] = df["date_click"].dt.minute
df["date_click_dayofweek"] = df["date_click"].dt.dayofweek

df["date_joined_year"] = df["date_joined"].dt.year
df["date_joined_month"] = df["date_joined"].dt.month
df["date_joined_week"] = df["date_joined"].dt.week
df["date_joined_day"] = df["date_joined"].dt.day
df["date_joined_hour"] = df["date_joined"].dt.hour
df["date_joined_minute"] = df["date_joined"].dt.minute
df["date_joined_dayofweek"] = df["date_joined"].dt.dayofweek

df["last_login_year"] = df["last_login"].dt.year
df["last_login_month"] = df["last_login"].dt.month
df["last_login_week"] = df["last_login"].dt.week
df["last_login_day"] = df["last_login"].dt.day
df["last_login_hour"] = df["last_login"].dt.hour
df["last_login_minute"] = df["last_login"].dt.minute
df["last_login_dayofweek"] = df["last_login"].dt.dayofweek

df["birthDate_year"] = df["birthDate"].dt.year
df["birthDate_month"] = df["birthDate"].dt.month
df["birthDate_week"] = df["birthDate"].dt.week
df["birthDate_day"] = df["birthDate"].dt.day
df["birthDate_hour"] = df["birthDate"].dt.hour
df["birthDate_minute"] = df["birthDate"].dt.minute
df["birthDate_dayofweek"] = df["birthDate"].dt.dayofweek

df["date_search_year"] = df["date_search"].dt.year
df["date_search_month"] = df["date_search"].dt.month
df["date_search_week"] = df["date_search"].dt.week
df["date_search_day"] = df["date_search"].dt.day
df["date_search_hour"] = df["date_search"].dt.hour
df["date_search_minute"] = df["date_search"].dt.minute
df["date_search_dayofweek"] = df["date_search"].dt.dayofweek
df = df.drop(['date_click','date_joined','last_login','birthDate','date_search'],axis=1)
df

,category,postcode,latitude,longitude,Text,date_click_year,date_click_month,date_click_week,date_click_day,date_click_hour,...,birthDate_hour,birthDate_minute,birthDate_dayofweek,date_search_year,date_search_month,date_search_week,date_search_day,date_search_hour,date_search_minute,date_search_dayofweek
0,electronics,54000,48.6913,6.18101,five,2020,11,47,17,15,...,18,53,0,2021.0,1.0,2.0,13.0,15.0,12.0,2.0
1,"Tech & Entertainment , mass media",54000,48.6913,6.18101,five,2020,11,47,17,15,...,18,53,0,2021.0,1.0,2.0,13.0,15.0,12.0,2.0
2,automobile,54000,48.6913,6.18101,five,2020,11,47,17,15,...,18,53,0,2021.0,1.0,2.0,13.0,15.0,12.0,2.0
3,automobile,54000,48.6913,6.18101,five,2020,11,47,17,15,...,18,53,0,2021.0,1.0,2.0,13.0,15.0,12.0,2.0
4,Music streaming service,54000,48.6913,6.18101,five,2020,11,47,17,15,...,18,53,0,2021.0,1.0,2.0,13.0,15.0,12.0,2.0
5,Telecommunications,54000,48.6913,6.18101,five,2020,11,47,17,15,...,18,53,0,2021.0,1.0,2.0,13.0,15.0,12.0,2.0
6,automobile,54000,48.6913,6.18101,five,2020,11,47,17,15,...,18,53,0,2021.0,1.0,2.0,13.0,15.0,12.0,2.0
7,Beverage,54000,48.6913,6.18101,five,2020,11,47,17,15,...,18,53,0,2021.0,1.0,2.0,13.0,15.0,12.0,2.0
8,Beverage,54000,48.6913,6.18101,five,2020,11,47,17,15,...,18,53,0,2021.0,1.0,2.0,13.0,15.0,12.0,2.0
9,Cosmetic products,54000,48.6913,6.18101,five,2020,11,47,17,15,...,18,53,0,2021.0,1.0,2.0,13.0,15.0,12.0,2.0


In [4]:
x_cols = [c for c in df.columns if c != 'category' ]
# set input matrix and target column
X = df[x_cols]
y = df['category']
# show first rows of data
df.head()
# data split train / test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=1234)
X_train
# fill missing values
train_mode = dict(X_train.mode().iloc[0])
X_train = X_train.fillna(train_mode)
print(train_mode)

{'postcode': '54000', 'latitude': 48.69127075918049, 'longitude': 6.181005123242174, 'Text': 'five', 'date_click_year': 2020, 'date_click_month': 11, 'date_click_week': 47, 'date_click_day': 17, 'date_click_hour': 15, 'date_click_minute': 45, 'date_click_dayofweek': 1, 'date_joined_year': 2020, 'date_joined_month': 11, 'date_joined_week': 46, 'date_joined_day': 14, 'date_joined_hour': 16, 'date_joined_minute': 42, 'date_joined_dayofweek': 5, 'last_login_year': 2021, 'last_login_month': 1, 'last_login_week': 3, 'last_login_day': 20, 'last_login_hour': 0, 'last_login_minute': 12, 'last_login_dayofweek': 2, 'birthDate_year': 2020, 'birthDate_month': 12, 'birthDate_week': 50, 'birthDate_day': 7, 'birthDate_hour': 18, 'birthDate_minute': 53, 'birthDate_dayofweek': 0, 'date_search_year': 2021.0, 'date_search_month': 1.0, 'date_search_week': 2.0, 'date_search_day': 13.0, 'date_search_hour': 15.0, 'date_search_minute': 12.0, 'date_search_dayofweek': 2.0}


In [5]:
# convert categoricals
encoders = {}
for column in ['Text']:
    categorical_convert = LabelEncoder()
    X_train[column] = categorical_convert.fit_transform(X_train[column])
    encoders[column] = categorical_convert

In [6]:
# train the Random Forest algorithm
rf = RandomForestClassifier(n_estimators = 100)
rf = rf.fit(X_train, y_train)

In [7]:
# train the Extra Trees algorithm
et = ExtraTreesClassifier(n_estimators = 100)
et = et.fit(X_train, y_train)

In [8]:
# save preprocessing objects and RF algorithm
joblib.dump(train_mode, "./train_modeN.joblib", compress=True)
joblib.dump(encoders, "./encodersN.joblib", compress=True)
joblib.dump(rf, "./random_forestN.joblib", compress=True)
joblib.dump(et, "./extra_treesN.joblib", compress=True)

['./extra_treesN.joblib']

In [9]:
try:
    # JSON to pandas DataFrame
    input_data = pd.DataFrame({
        "postcode": "54000", 
        "Text": "five", 
        "latitude": 48.69127075918047, 
        "longitude": 6.1810051232421745, 
        "date_click_year": 2020, 
        "date_click_month": 11, 
        "date_click_week": 47, 
        "date_click_day": 17, 
        "date_click_hour": 15, 
        "date_click_minute": 45, 
        "date_click_dayofweek": 1, 
        "date_joined_year": 2020, 
        "date_joined_month": 11, 
        "date_joined_week": 46, 
        "date_joined_day": 14, 
        "date_joined_hour": 16, 
        "date_joined_minute": 42, 
        "date_joined_dayofweek": 5, 
        "last_login_year": 2021, 
        "last_login_month": 1, 
        "last_login_week": 53, 
        "last_login_day": 2, 
        "last_login_hour": 20, 
        "last_login_minute": 23, 
        "last_login_dayofweek": 5, 
        "birthDate_year": 2020, 
        "birthDate_month": 12, 
        "birthDate_week": 50, 
        "birthDate_day": 7, 
        "birthDate_hour": 18, 
        "birthDate_minute": 53, 
        "birthDate_dayofweek": 0, 
        "date_search_year": 2021, 
        "date_search_month": 1, 
        "date_search_week": 2, 
        "date_search_day": 13, 
        "date_search_hour": 15, 
        "date_search_minute": 12, 
        "date_search_dayofweek": 2
    }, index=[0])
    # fill missing values
    input_data.fillna(train_mode)
    # convert categoricals
    for column in ["Text",]:
        categorical_convert = encoders[column]
        input_data[column] = categorical_convert.transform(input_data[column])

    prediction = rf.predict(input_data)[0]


    label = prediction
    prediction = {"probability":None, "label": label, "status": "OK"}
    print(prediction)
except Exception as e:
    print({"status": "Error", "message": str(e)})
        

{'probability': None, 'label': 'automobile', 'status': 'OK'}
